# Bookworm


## Overview

In this project, you will build a simple question-answering agent that is able to learn from any text data you provide, and answer queries posed in natural language. You will use IBM Watson's cloud-based services to process the input text data and find relevant responses.

## Learning Objectives

By completing this project, you will learn how to:

- Create a cloud-based NLP service instance and configure it.
- Ingest a set of text documents using the service and analyze the results.
- Accept questions in natural language and parse them.
- Find relevant answers from the preprocessed text data.

## Getting Started

In order to use Watson's cloud-based services, you first need to create an account on the [IBM Bluemix platform](https://console.ng.bluemix.net/).

<div>
    <div style="display: table-cell; width: 50%;">
        <img src="images/watson-logo.png" alt="IBM Watson logo" width="200" />
    </div>
    <div style="display: table-cell; width: 50%;">
        <img src="images/bluemix-logo.png" alt="IBM Bluemix logo" width="400" />
    </div>
</div>

Then, for each service you want to use, you have to create an instance of that service. You can continue with the tasks below, and create a service instance when indicated.

## 1. Create and configure Discovery service

Create an instance of the **Discovery** service. You will use this to process a set of text documents, and _discover_ relevant facts and relationships.

- Go to the [IBM Bluemix Catalog](https://console.ng.bluemix.net/catalog/?taxonomyNavigation=services&category=watson).
- Select the service you want, **Discovery**, under the **Watson** category.
- Enter a Service Name for that instance, e.g. **Disco1** and a Credential Name, e.g. **Disco1-Creds** (these are just for you to be able to refer to later, they do not affect the functioning of the service).
- You should be able to see your newly-created service in your [Services Dashboard](https://console.ng.bluemix.net/dashboard/services).
- Open the service instance, click on the **Service credentials** tab, and then **View credentials** under Actions. This is where you will find the username and password to use when connecting to the service.

<img src="images/discovery-creds.png" alt="Discovery Service - Credentials tab" width="800" />

Save the credentials for the discovery service in a JSON file in the current directory named `service-credentials.json` with the following format:

```json
{
    "discovery": {
        "username": "<your Discovery username here>",
        "password": "<your Discovery password here>"
    },
    "conversation": {
        "username": "",
        "password": ""
    }
}

```

You will be filling out the Conversation service credentials later, when you create an instance for it. Note that you should keep these credentials secret. Please do not turn them in with your submission!

### Connect to the service instance

Let's connect to the service instance you just created using IBM Watson's [Python SDK](https://github.com/watson-developer-cloud/python-sdk). You will first need to install the SDK:
```bash
pip install watson-developer-cloud
```

Now execute each code cell below using **`Shift+Enter`**, and complete any steps indicated by a **`TODO`** comment. For more information on the Discovery service, please read the [Documentation](https://www.ibm.com/watson/developercloud/doc/discovery/index.html) and look at the [API Reference](https://www.ibm.com/watson/developercloud/discovery/api/v1/?python) as needed.

In [2]:
# Usual Python imports
import sys
import os
import glob
import json

# BeautifulSoup, for parsing HTML
from bs4 import BeautifulSoup

# Matplotlib, for plotting
import matplotlib.pyplot as plt
%matplotlib inline

# Watson Python SDK
import watson_developer_cloud

# Utility functions
import helper

In [3]:
# Connect to the Discovery service instance
# TODO: Ensure that your username and password from the Service Credentials tab are in service-credentials.json
# Note that these credentials are different from your IBM Bluemix login, and are specific to the service instance
discovery_creds = helper.fetch_credentials('discovery')
discovery = watson_developer_cloud.DiscoveryV1(
    version='2016-11-07',
    username=discovery_creds['username'],
    password=discovery_creds['password'])

### Create an environment

The Discovery service organizes everything needed for a particular application in an _environment_. Let's create one called "Bookworm" for this project.

> _**Note**: It is okay to run this block multiple times - it will not create duplicate environments with the same name._

In [4]:
# Prepare an environment to work in
env, env_id = helper.fetch_object(
    discovery, "environment", "Bookworm",
    create=True, create_args=dict(
        description="A space to read and understand stories",  # feel free to edit
        size=0  # use 0 for free plan (see API reference for more on sizing)
    ))
print(json.dumps(env, indent=2))

Found environment: Bookworm (c8bad134-8b1a-449b-a406-ddbbc9870ade)
{
  "status": "active",
  "name": "Bookworm",
  "environment_id": "c8bad134-8b1a-449b-a406-ddbbc9870ade",
  "description": "A space to read and understand stories",
  "read_only": false,
  "updated": "2017-06-15T01:08:48.095Z",
  "created": "2017-06-15T01:08:48.095Z",
  "index_capacity": {
    "disk_usage": {
      "total_bytes": 4294967296,
      "used_bytes": 0,
      "used": "0 KB",
      "total": "4 GB",
      "percent_used": 0
    },
    "memory_usage": {
      "total_bytes": 1056309248,
      "used_bytes": 142799760,
      "used": "136.18 MB",
      "total": "1007.38 MB",
      "percent_used": 13.52
    }
  },
  "size": 0
}


### Verify configuration options

A _configuration_ defines what natural language processing routines are applied to any documents that are submitted to the service. Each environment gets a default configuration when it is created.

You can fetch the default configuration and view the different options using the following piece of code.


In [5]:
# View default configuration
cfg_id = discovery.get_default_configuration_id(environment_id=env_id)
cfg = discovery.get_configuration(environment_id=env_id, configuration_id=cfg_id)
print(json.dumps(cfg, indent=2))

{
  "name": "Default Configuration",
  "description": "The configuration used by default when creating a new collection without specifying a configuration_id.",
  "configuration_id": "ed8ca644-4ef6-4fd1-856c-6128676f9c7d",
  "updated": "2017-06-15T01:08:48.122Z",
  "normalizations": [],
  "created": "2017-06-15T01:08:48.122Z",
  "enrichments": [
    {
      "options": {
        "extract": "keyword, entity, doc-sentiment, taxonomy, concept, relation",
        "sentiment": true,
        "quotations": true
      },
      "source_field": "text",
      "destination_field": "enriched_text",
      "enrichment": "alchemy_language"
    }
  ],
  "conversions": {
    "html": {
      "exclude_content": {
        "xpaths": []
      },
      "exclude_tags_completely": [
        "script",
        "sup"
      ],
      "exclude_tag_attributes": [
        "EVENT_ACTIONS"
      ],
      "keep_content": {
        "xpaths": []
      },
      "exclude_tags_keep_content": [
        "font",
        "em",
    

There are 3 main configuration blocks that affect how input documents are processed:
 
1. **`conversions`**: How to convert documents in various formats (Word, PDF, HTML) and extract elements that indicate some structure (e.g. headings).
2. **`enrichments`**: What NLP output results are we interested in (keywords, entities, sentiment, etc.).
3. **`normalizations`**: Post-processing steps to be applied to the output. This can be left empty in most cases, unless you need the output to be normalized into a very specific format.

_**Note**: The default configuration for an environment cannot be modified. If you need to change any of the options, you will need to create a new one, and then edit it. The easiest way to do this is using the service dashboard, which is described later._

### Test your configuration

It is a good idea to test your configuration on a small sample text before you apply it to a larger document collection.

_**Note**: We have supplied a sample document (`data/sample.html`) containing the opening crawl text for Star Wars: Episode IV, but you are free to use a text of your choosing._

**Q**: (optional) If you use your own sample text, provide a brief title and description below.

**A**: 

In [6]:
# Test configuration on some sample text
data_dir = "data"
filename = os.path.join(data_dir, "sample.html")
with open(filename, "r") as f:
    res = discovery.test_document(environment_id=env_id, configuration_id=cfg_id, fileinfo=f)
print(json.dumps(res, indent=2))

{
  "status": "completed",
  "enriched_field_units": 1,
  "notices": [],
  "original_media_type": "text/html",
  "snapshots": [
    {
      "step": "html_input",
      "snapshot": {
        "html": "<html>\n<head>\n    <title>Star Wars: Episode IV - A New Hope (Opening Crawl)</title>\n</head>\n<body>\n    <article>\n        <h1>Star Wars: Episode IV - A New Hope (Opening Crawl)</h1>\n        <p>\n            It is a period of civil war. Rebel spaceships, striking from a hidden base, have won their first victory against the evil Galactic Empire.\n        </p><p>\n            During the battle, Rebel spies managed to steal secret plans to the Empire's ultimate weapon, the DEATH STAR, an armored space station with enough power to destroy an entire planet.\n        </p><p>\n            Pursued by the Empire's sinister agents, Princess Leia races home aboard her starship, custodian of the stolen plans that can save her people and restore freedom to the galaxy...\n        </p>\n    </article

### Analyze test output

The results returned by the service contain a _snapshot_ of the information extracted at each step of processing - document conversions, enrichments and normalizations. We are interested in the output of applying enrichments (`"enrichments_output"`) or after normalizing them (`"normalizations_output"`). These should be identical if no post-processing/normalizations were specified in the configuration.

In [7]:
# Take a closer look at the results from the "enrichments_output" or "normalizations_output" step
output = next((s["snapshot"] for s in res["snapshots"] if s["step"] == "enrichments_output"), None)
print(json.dumps(output, indent=2))

{
  "enriched_text": {
    "status": "OK",
    "relations": [
      {
        "subject": {
          "text": "It"
        },
        "action": {
          "lemmatized": "be",
          "verb": {
            "tense": "present",
            "text": "be"
          },
          "text": "is"
        },
        "sentence": " It is a period of civil war.",
        "object": {
          "sentiment": {
            "type": "negative",
            "mixed": false,
            "score": -0.564509
          },
          "text": "a period of civil war",
          "keywords": [
            {
              "text": "civil war"
            },
            {
              "text": "period"
            }
          ],
          "entities": [
            {
              "type": "FieldTerminology",
              "text": "civil war"
            }
          ]
        }
      },
      {
        "subject": {
          "text": "Rebel spaceships, striking from a hidden base,",
          "keywords": [
            {
   

Answer the following questions based on the output above. Note that it contains the input HTML, extracted text and metadata as well as the actual enrichment results (`"enriched_text"` block).

#### Sentiment

**Q**: What is the overall sentiment detected in this text? Mention the `type` (positive/negative) and `score`.<br />
(_Hint: Look for the `"docSentiment"` key in the output._)

**A**: The document sentiment is slightly negative indicated by the score of -0.57. 


#### Concepts

**Q**: List 3 concepts that have been identified with a relevance > 0.5. Note that not all concepts here may be present directly in the text, some may have been inferred by Watson.<br />
(_Hint: Look for `"concepts"`._)

**A**:
Luke Skywalker and Darth Vader are 2 concepts that have been inferred by Watson as realted to the text. Star Wars is a concept that has been identified as a concepts and it exists already in the text. 


#### Relations

Each relation is essentially a deeper analysis of a sentence (or part of a sentence). Here is a sample relation:
```json
{
  "sentence": " During the battle, Rebel spies managed to steal secret plans to the Empire's ultimate weapon, the DEATH STAR, an armored space station with enough power to destroy an entire planet.",
  "subject": {
    "text": "Rebel spies",
    "keywords": [
      {
        "text": "Rebel spies"
      }
    ]
  },
  "action": {
    "text": "managed to steal",
    "lemmatized": "manage to steal",
    "verb": {
      "text": "steal",
      "tense": "future"
    }
  },
  "object": {
    "text": "secret plans",
    "keywords": [
      {
        "text": "secret plans"
      }
    ]
  }
 }
```

In this case, Watson seems to have done a pretty good job of extracting some meaning from the sentence.

**Q**: Find a relation where the extracted meaning is not as accurate, or not what you would've expected. List the `sentence`, `subject`, `action` and `object` parts as identified, and what you would've marked instead.<br />
(_Hint: Look for `"relations"`._)

**A**: The same sentence "During the battle, Rebel spies managed to steal secret plans to the Empire's ultimate weapon, the DEATH STAR, an armored space station with enough power to destroy an entire planet." has been analyzed with a false result also. Watson seems to have inferred the action as "destroy". However, in this case the action was "to steal"; stealing of the plans. 
{
        "action": {
          "lemmatized": "to destroy",
          "text": "to destroy",
          "verb": {
            "text": "destroy",
            "tense": "future"
          }
        },
        "subject": {
          "text": "Rebel spies",
          "keywords": [
            {
              "text": "Rebel spies"
            }
          ]
        },
        "sentence": " During the battle, Rebel spies managed to steal secret plans to the Empire's ultimate weapon, the DEATH STAR, an armored space station with enough power to destroy an entire planet.",
        "object": {
          "text": "an entire planet",
          "sentiment": {
            "type": "positive",
            "mixed": false,
            "score": 0.748639
          },
          "keywords": [
            {
              "text": "entire planet"
            }
          ],
          "sentimentFromSubject": {
            "type": "negative",
            "mixed": false,
            "score": -0.754578
          }
        }

#### Keywords

You may have noticed that Watson identifies some `"keywords"` in the relations, e.g. `"Rebel spies"` and `"secret plans"` in the Star Wars example above. The output also contains a list of all keywords at the top level, for your convenience, along with their relevance to the document and sentiment conveyed. Let's visualize these keywords as a word cloud!

Note: We'll be using this handy [worldcoud library](https://github.com/amueller/word_cloud) to generate the visualization. So you will need to install it first:
```bash
pip install wordcloud
```

In [8]:
# Visualize keywords by relevance as a wordcloud
from wordcloud import WordCloud

wc_data = { w["text"]: w["relevance"] for w in output["enriched_text"]["keywords"] }
wc = WordCloud(width=400, height=300, scale=2, background_color="white", colormap="Vega10")
wc.generate_from_frequencies(wc_data)  # use precomputed relevance instead of frequencies

plt.figure(figsize=(4, 3), dpi=200)
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")

(-0.5, 799.5, 599.5, -0.5)

Feel free to play with this visualization and improve it. What about using a different metric instead of relevance, e.g. direct word frequencies that the wordcloud library computes by default?

#### Other results

Watson's output also includes processed results from other enrichments that were applied to the text, including entities and taxonomy (what topic or category does this text relate to).

```json
"taxonomy": [
  {
    "label": "/art and entertainment/movies and tv/movies",
    "score": 0.584247,
    "confident": false
  },
  {
    "label": "/society/unrest and war",
    "score": 0.517031,
    "confident": false
  },
  {
    "confident": false,
    "label": "/law, govt and politics/armed forces/army",
    "score": 0.215561
  }
],
```

Get a good sense of all the different pieces of information available in the results. Start thinking about which ones will be useful for looking up answers to questions, and how you might use them.

## 2. Ingest documents

### Create a collection

A _collection_ is used to organize documents of the same kind. For instance, you may want to create a collection of book reviews, or a collection of Wikipedia articles, but it may not make much sense to mix the two groups. This allows Watson to make meaningful inferences over the set of documents, find commonalities and identify important concepts.

Let's create one called "Story Chunks".

In [9]:
# Prepare a collection of documents to use
col, col_id = helper.fetch_object(discovery, "collection", "Story Chunks", environment_id=env_id,
    create=True, create_args=dict(
        environment_id=env_id, configuration_id=cfg_id,
        description="Stories and plots split up into chunks suitable for answering"
    ))
print(json.dumps(col, indent=2))

Created collection: Story Chunks (0c7e28d1-d5ad-4a73-b6c3-83290a84c553)
{
  "collection_id": "0c7e28d1-d5ad-4a73-b6c3-83290a84c553",
  "name": "Story Chunks",
  "description": "Stories and plots split up into chunks suitable for answering",
  "configuration_id": "ed8ca644-4ef6-4fd1-856c-6128676f9c7d",
  "updated": "2017-06-15T01:30:02.602Z",
  "created": "2017-06-15T01:30:02.602Z",
  "status": "active",
  "language": "en"
}


Once you have created a collection, you should be able to view it using the Discovery Service tool. To open, go to the **Manage** tab for your service instance, and click the **Launch tool** button.

<img src="images/discovery-manage.png" alt="Discovery service - Manage tab" width="800" />


Here you should see the "Story Chunks" collection you just created.

<img src="images/discovery-tooling.png" alt="Discovery service - Tool showing collections" width="800" />

You can open the collection to view more details about it. If you need to modify configuration options, click the **Switch** link and create a new configuration (the default one cannot be changed).

### Add documents

Okay, now that we have everything set up, let's add a set of "documents" we want Watson to look up answers from, using the Python SDK. Note that Watson treats each "document" as a unit of text that is returned as the result of a query. But we want to retrieve a paragraph of text for each question. So, let's split each file up into individual paragraphs. We will use the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library for this purpose.

_**Note**: You could also add and manage documents in the collection using the Discovery tool, but you would have to split paragraphs up into separate files._

_**Note**: We have provided a set of files (`data/Star-Wars/*.html`) with summary plots for Star Wars movies, but you are free to use a collection of your choice. Open one of the files in a text editor to see how the paragraphs are delimited using `<p>...</p>` tags - this is how the code block below split paragraphs into separate "documents"._

In [10]:
# Add documents to collection
doc_ids = []  # to store the generated id for each document added
for filename in glob.glob(os.path.join(data_dir, "Star-Wars", "*.html")):
    print("Adding file:", filename)
    with open(filename, "r") as f:
        # Split each individual <p> into its own "document"
        doc = f.read()
        soup = BeautifulSoup(doc, 'html.parser')
        for i, p in enumerate(soup.find_all('p')):
            doc_info = discovery.add_document(environment_id=env_id, collection_id=col_id,
                file_data=json.dumps({"text": p.get_text(strip=True)}),
                mime_type="application/json",
                metadata={"title": soup.title.get_text(strip=True)})
            doc_ids.append(doc_info["document_id"])
print("Total", len(doc_ids), "documents added.")

Adding file: data/Star-Wars/Episode-I_The-Phantom-Menace.html
Adding file: data/Star-Wars/Episode-II_Attack-of-the-Clones.html
Adding file: data/Star-Wars/Episode-III_Revenge-of-the-Sith.html
Adding file: data/Star-Wars/Episode-IV_A-New-Hope.html
Adding file: data/Star-Wars/Episode-V_The-Empire-Strikes-Back.html
Adding file: data/Star-Wars/Episode-VI_Return-of-the-Jedi.html
Adding file: data/Star-Wars/Episode-VII_The-Force-Awakens.html
Adding file: data/Star-Wars/Rogue-One.html
Total 42 documents added.


If you look at the collection details, you may notice that the `"document_counts"` field now shows some documents as `available` or `processing`. Once processing is complete, you should see all the documents under the `available` count.

In [11]:
# View collection details to verify all documents have been processed
col, col_id = helper.fetch_object(discovery, "collection", "Story Chunks", environment_id=env_id)
print(json.dumps(col, indent=2))

Found collection: Story Chunks (0c7e28d1-d5ad-4a73-b6c3-83290a84c553)
{
  "collection_id": "0c7e28d1-d5ad-4a73-b6c3-83290a84c553",
  "name": "Story Chunks",
  "training_status": {
    "sufficient_label_diversity": false,
    "notices": 0,
    "minimum_examples_added": false,
    "total_examples": 0,
    "data_updated": "",
    "successfully_trained": "",
    "processing": false,
    "minimum_queries_added": false,
    "available": false
  },
  "description": "Stories and plots split up into chunks suitable for answering",
  "configuration_id": "ed8ca644-4ef6-4fd1-856c-6128676f9c7d",
  "updated": "2017-06-15T01:30:02.602Z",
  "created": "2017-06-15T01:30:02.602Z",
  "status": "active",
  "language": "en",
  "document_counts": {
    "processing": 0,
    "failed": 0,
    "available": 42
  }
}


So, what did the Discovery service learn? If you list the fields extracted from the set of documents in the collection as part of the enrichment process, you'll see familiar fields like `concepts`, `entities` and `keywords` that were returned in the test analysis.

In [12]:
# List all fields extracted
discovery.list_collection_fields(environment_id=env_id, collection_id=col_id)

{'fields': [{'field': '0c7e28d1-d5ad-4a73-b6c3-83290a84c553.mappings.document._all.analyzer',
   'type': 'english'},
  {'field': 'enriched_text', 'type': 'nested'},
  {'field': 'extracted_metadata', 'type': 'nested'},
  {'field': 'metadata', 'type': 'nested'},
  {'field': 'text', 'type': 'string'},
  {'field': 'text.analyzer', 'type': 'english'},
  {'field': 'enriched_text.properties.concepts', 'type': 'nested'},
  {'field': 'enriched_text.properties.docSentiment', 'type': 'nested'},
  {'field': 'enriched_text.properties.entities', 'type': 'nested'},
  {'field': 'enriched_text.properties.keywords', 'type': 'nested'},
  {'field': 'enriched_text.properties.language', 'type': 'string'},
  {'field': 'enriched_text.properties.language.analyzer', 'type': 'english'},
  {'field': 'enriched_text.properties.relations', 'type': 'nested'},
  {'field': 'enriched_text.properties.status', 'type': 'string'},
  {'field': 'enriched_text.properties.status.analyzer', 'type': 'english'},
  {'field': 'enric

### Test query

Let's perform a simple query to see if the service can fetch the proper document for us:
> _Look for all paragraphs that have a relation (sentence) with "Jar Jar" as the subject, and return the title and text._


In [14]:
# A simple query
results = discovery.query(environment_id=env_id, collection_id=col_id,
    query_options={
        "query": "enriched_text.relations.subject.text:\"Death Star\"",
        "return": "metadata.title,text"
    })
print(json.dumps(results, indent=2))

{
  "matching_results": 3,
  "results": [
    {
      "text": "Krennic corners Jyn, declaring the Empire's victory, but Cassian, who has survived, shoots and wounds Krennic. Jyn transmits the schematics to the Rebel command ship. The Death Star enters orbit above Scarif, where Tarkin uses another low-power shot from the superlaser to destroy the Empire's base. Krennic is killed instantly, while Jyn and Cassian embrace on a beach before dying in the blastwave.",
      "id": "00bc6c36-b911-4b12-977b-9b3f588b3982",
      "score": 4.7950697,
      "metadata": {
        "title": "Star Wars: Rogue One"
      }
    },
    {
      "text": "Upon the Falcon's arrival at the location of Alderaan, the group discover that the planet has been destroyed by order of the Death Star's commanding officer, Grand Moff Tarkin, as a show of power. The Falcon is captured by the Death Star's tractor beam and brought into its hangar bay. While Obi-Wan goes to disable the tractor beam, Luke discovers that Leia i

Change the above query and see what results you get! Try to find one that returns relevant results, and keep that (along with the output) for review.

> See [Query building reference](https://www.ibm.com/watson/developercloud/doc/discovery/query-reference.html) for descriptions of all possible parameters, operators and aggregations. You can also choose to build the query using the web interface (click the "Story Chunks" collection to query it), and then reproduce the query here.

Then answer the questions below:

**Q**: What query did you try? Express it in plain words below.

**A**: 1. Tried "Death star" keyword. 
2. Tried "fictional aviators" concept query in the GUI: enriched_text.concepts.text:"Fictional sword fighters"


**Q**: What answer did you get back from Watson? You only need to mention the relevant snippet of text fro mthe paragraph(s) returned.

**A**: 1.Returned two paragraphs that had the keyword in them. 
2. Returned 2 documents with sword fighting characters. 


## 3. Parse natural language questions

In order to understand questions posed in natural language, we'll use another Watson service called [Conversation](https://www.ibm.com/watson/developercloud/doc/conversation/index.html). It can be used to design conversational agents or _chatbots_ that exhibit complex behavior, but for the purpose of this project, we'll only use it to parse certain kinds of queries.

### Create a Conversation service instance

Just like you did for the Discovery service, create an instance of the Conversation service. Then launch the associated tool from the service dashboard.

<img src="images/conversation-tooling.png" alt="Conversation service - Tool homepage" width="800" />

A _workspace_ allows you to keep all the items you need for a particular application in one place, just like an _environment_ in case of the Discovery service. Create one called "Bookworm" with a suitable description, such as "I know a lot of stories. Ask me a question!"

<img src="images/conversation-workspace.png" alt="Conversation service - Blank workspace" width="800" />

This should open up a blank workspace, where you can add intents, define the entities you want the agent to idenitfy and structure the overall dialog.

### Add intents

An _intent_ is the goal or purpose of a user's input. Create a set of intents (at least 3) that capture the different kinds of questions that you want the system to answer, e.g. _who_, _what_ and _where_. Along with each intent, add a list of user examples or _utterances_ that map to that intent.

For instance, you could enter the following examples for the _where_ intent:

- Where is the Jedi temple located?
- Where was Luke born?

The Conversation service recommends at least 5 examples for each intent so that Watson learns how to recognize it. These don't have to be very precise, but more examples the better.

<img src="images/conversation-intents.png" alt="Conversation service - Intents listed" width="800" />

Feel free to create your own intents, based on the kinds of questions you want the system to answer, e.g. "How many ...", "What are the most common ..." etc. Each intent will need to be mapped to an appropriate query.

> See [**Defining intents**](https://www.ibm.com/watson/developercloud/doc/conversation/intents.html) for a helpful video and further instructions.

**Q**: What intents did you add to the Conversation service instance?

**A**: Added: #When, #Which, #Result, #Who intents. 



**Q**: Pick one of these intents, and list at least 5 examples for the intent that you entered.

**A**: #Result
What happened as a result of the war?
Did droids win the war?
Did the death star explode?
Was Darth Vader killed?
What happened in the end?



### Add entities

Once you have your intents set, let's tell the service what entities we want it to identify. One way to do this is using the tool interface, and entering them one-by-one.

> Go to [**Defining entities**](https://www.ibm.com/watson/developercloud/doc/conversation/entities.html) to see how that is done.

But that can be tedious! So let's refer back to the entities that the Discovery service identified, and load them in programmatically.

As before, let's connect to the Conversation service first. Remember to enter your service credentials below.

In [15]:
# Connect to the Conversation service instance
# TODO: Enter your username and password from the Service Credentials tab in service-credentials.json
conversation_creds = helper.fetch_credentials('conversation')
conversation = watson_developer_cloud.ConversationV1(
    version='2017-02-03',
    username=conversation_creds['username'],
    password=conversation_creds['password'])

Fetch the workspace you just created called "Bookworm".

In [16]:
wrk, wrk_id = helper.fetch_object(conversation, "workspace", "Bookworm")
print(json.dumps(wrk, indent=2))

Found workspace: Bookworm (30c56eb5-0806-493f-bd5d-17c9e9894658)
{
  "status": "Available",
  "name": "Bookworm",
  "description": "Ask questions about stories",
  "updated": "2017-06-15T01:48:05.401Z",
  "workspace_id": "30c56eb5-0806-493f-bd5d-17c9e9894658",
  "created": "2017-06-15T01:45:22.286Z",
  "language": "en",
  "metadata": null
}


Collect all the entities from the Discovery service collection.

In [17]:
# Get all the entities from the collection and group them by type
response = discovery.query(environment_id=env_id, collection_id=col_id,
    query_options={
        "return": "enriched_text.entities.type,enriched_text.entities.text"
    })

# Group individual entities by type ("Person", "Location", etc.)
entities_by_type = {}
for document in response["results"]:
    for entity in document["enriched_text"]["entities"]:
        if entity["type"] not in entities_by_type:
            entities_by_type[entity["type"]] = set()
        entities_by_type[entity["type"]].add(entity["text"])

# Ignore case to avoid duplicates
for entity_type in entities_by_type:
    entities_by_type[entity_type] = {
        e.lower(): e for e in entities_by_type[entity_type]
    }.values()

# Restructure for loading into Conversation workspace
entities_grouped = [{
    "entity": entity_type,
    "values": [{"value": entity} for entity in entities]}
        for entity_type, entities in entities_by_type.items()]
entities_grouped


[{'entity': 'Person',
  'values': [{'value': 'Shmi'},
   {'value': 'Mustafar'},
   {'value': 'Padmé Amidala'},
   {'value': 'Beru Lars'},
   {'value': 'Amidala'},
   {'value': 'Polis Massa'},
   {'value': 'Geonosis'},
   {'value': 'Queen Amidala'},
   {'value': 'Palpatine'},
   {'value': 'Darth Sidious'},
   {'value': 'Gunray'},
   {'value': 'Yoda'},
   {'value': 'Princess Leia'},
   {'value': 'Darth Vader'},
   {'value': 'Jango Fett'},
   {'value': 'Boss Nass'},
   {'value': 'Anakin'},
   {'value': 'Organa'},
   {'value': 'Anakin Skywalker'},
   {'value': 'Viceroy Nute Gunray'},
   {'value': 'Owen'},
   {'value': 'Senator Padmé Amidala'},
   {'value': 'Leia'},
   {'value': 'Senator Palpatine'},
   {'value': 'Zam Wesell'},
   {'value': 'Boba'},
   {'value': 'Luke'},
   {'value': 'Vader'},
   {'value': 'Mace Windu'},
   {'value': 'Qui-Gon'},
   {'value': 'Darth Maul'},
   {'value': 'Chancellor Valorum'},
   {'value': 'Padmé'},
   {'value': 'Cliegg Lars'},
   {'value': 'Supreme Chancello

Update the workspace with these entities and verify that have been added correctly.

In [18]:
# Add these grouped entities to the Conversation workspace
conversation.update_workspace(workspace_id=wrk_id, entities=entities_grouped)

workspace_details = conversation.get_workspace(workspace_id=wrk_id, export=True)
print(json.dumps(workspace_details["entities"], indent=2))

[
  {
    "type": null,
    "description": null,
    "values": [
      {
        "value": "Federation Viceroy Nute Gunray",
        "created": "2017-06-15T01:53:00.507Z",
        "synonyms": [],
        "updated": "2017-06-15T01:53:00.507Z",
        "metadata": null
      }
    ],
    "source": null,
    "entity": "Company",
    "created": "2017-06-15T01:53:00.507Z",
    "updated": "2017-06-15T01:53:00.507Z",
    "open_list": null
  },
  {
    "type": null,
    "description": null,
    "values": [
      {
        "value": "commander",
        "created": "2017-06-15T01:53:00.507Z",
        "synonyms": [],
        "updated": "2017-06-15T01:53:00.507Z",
        "metadata": null
      },
      {
        "value": "representative",
        "created": "2017-06-15T01:53:00.507Z",
        "synonyms": [],
        "updated": "2017-06-15T01:53:00.507Z",
        "metadata": null
      },
      {
        "value": "Chancellor",
        "created": "2017-06-15T01:53:00.507Z",
        "synonyms": [],
  

_**Note**: Ensure that at least 3 entity types, with at least 1 example entity each have been added._

Here is what the list of entities should look like through the Conversation tool.

<img src="images/conversation-entities.png" alt="Conversation service - Entities listed" width="800" />

**Q**: Name 3 entity types that were added, with at least 1 example entity each (e.g. entity type: _City_, example: _Los Angeles_).

**A**: @Country: Naboo
@Facility: Jedi Temple
@Job TItle: Commander, representative, chancellor

### Design dialog flow

As a final step in creating the Conversation interface, let's design a typical dialog with a user. The most intuitive way to do this is to use the Dialog tab in the tool. Here, you can add _nodes_ that capture different stages in the dialog flow, and connect them in a meaningful way.

Go ahead and add at least 3 dialog nodes. Specify the triggers in terms of the intents and entities that you'd like to match, and an optional intermediate response like "Let me find that out for you." The actual response will be fetched by querying the Discovery service.

Here is what the dialog nodes should look like.

<img src="images/conversation-dialog_nodes.png" alt="Conversation service - Dialog nodes" width="640" />

**Q**: Specify 3 dialog nodes you added, along with the trigger (intent and/or entities) for each.

**A**: Added Who, Location and Result nodes for conversation. Triggers are as below:
Who: #who intent
Location: #where intent
result: #result intent

### Test dialog

Let's run through a test dialog to demonstrate how the system transitions to one of the nodes you defined above.

In [26]:
# Testing the dialog flow

# Start conversation with a blank message
results = conversation.message(workspace_id=wrk_id, message_input={})
context = results["context"]

# Then ask a sample question
question= "Who is Luke's father?"
results = conversation.message(workspace_id=wrk_id, message_input={
    "text": question,
    "context": context
})
print(json.dumps(results, indent=2))

{
  "context": {
    "conversation_id": "9d5901b7-18ee-4342-956d-9c6c44f08373",
    "system": {
      "dialog_turn_counter": 1,
      "_node_output_map": {
        "Anything else": [
          0
        ]
      },
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ],
      "branch_exited": true,
      "dialog_request_counter": 1,
      "branch_exited_reason": "completed"
    }
  },
  "output": {
    "text": [
      "I didn't understand. You can try rephrasing."
    ],
    "nodes_visited": [
      "Anything else"
    ],
    "log_messages": []
  },
  "intents": [
    {
      "intent": "who",
      "confidence": 1
    }
  ],
  "input": {
    "context": {
      "conversation_id": "550a3197-4dbf-4d2a-b085-49bb96e5d04a",
      "system": {
        "dialog_stack": [
          {
            "dialog_node": "Welcome"
          }
        ],
        "dialog_request_counter": 1,
        "dialog_turn_counter": 1,
        "_node_output_map": {
          "Welcome": [
    

## 4. Query document collection to fetch answers

The Discovery service includes a simple mechanism to make queries against your enriched collection of documents. But you have a lot of control over what fields are searched, how results are aggregated and values are returned.

### Process sample question

Choose a sample nautal language question to ask, and run it through the Conversation service, just like you did above when testing dialog flow.

In [74]:
# TODO: Run a sample question through Conversation service

# Testing the dialog flow
# Then ask a sample question
question= "Who vows to train Anakin"
results = conversation.message(workspace_id=wrk_id, message_input={
    "text": question,
    "context": context
})
print(json.dumps(results, indent=2))

{
  "context": {
    "conversation_id": "3d024718-d743-4852-b75e-573af8453496",
    "system": {
      "dialog_turn_counter": 1,
      "_node_output_map": {
        "node_6_1497492076127": [
          0
        ]
      },
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ],
      "branch_exited": true,
      "dialog_request_counter": 1,
      "branch_exited_reason": "completed"
    }
  },
  "output": {
    "text": [
      "Ok, let me see who indeed..."
    ],
    "nodes_visited": [
      "who"
    ],
    "log_messages": []
  },
  "intents": [
    {
      "intent": "who",
      "confidence": 0.8344757556915283
    }
  ],
  "input": {
    "context": {
      "conversation_id": "9ea8651f-42c9-40fd-b04f-3d1ea7394ac9",
      "system": {
        "dialog_stack": [
          {
            "dialog_node": "Welcome"
          }
        ],
        "dialog_request_counter": 1,
        "dialog_turn_counter": 1,
        "_node_output_map": {
          "Welcome": [
      

Now extract the intent and entities identified in the question, and optionally what dialog node was triggered (in case you need it later to customize your response). Some sample code is provided below, but you may need to modify it.

In [75]:
# TODO: Identify the intent(s) the user expressed (typically a single one)
query_intents = [intent["intent"] for intent in results["intents"]]
print("Intent(s):", query_intents)

# TODO: Extract the entities found in the question text
query_entities = [entity["value"] for entity in results["entities"]]
print("Entities:", query_entities)

# TODO: (optional) Find out what dialog node was triggered


Intent(s): ['who']
Entities: ['Anakin']


### Query the collection

Design a query based on the information extracted above, and run it against the document collection. The sample query provided below simple looks for all the entities in the raw `text` field. Modify it to suit your needs.

Take a look at the [API Reference](https://www.ibm.com/watson/developercloud/discovery/api/v1/?python#query-collection) to learn more about the query options available, and for more guidance see this [documentation page](https://www.ibm.com/watson/developercloud/doc/discovery/using.html).

_**Note**: You may want to design different queries based on the intent / dialog node that was triggered._

In [76]:
# TODO: Query the Discovery service based on the intent and entities
print(query_entities)
query_results = discovery.query(environment_id=env_id, collection_id=col_id,
    query_options={
        "query": "text:{}".format(",".join("\"{}\"".format(e) for e in query_entities)),
        "return": ""
    })
print(json.dumps(query_results, indent=2))

['Anakin']
{
  "matching_results": 14,
  "results": [
    {
      "extracted_metadata": {
        "sha1": "7e8cf0b6e7d6d9d24d6802d0ab969f4f42a38f6b",
        "file_type": "json",
        "filename": "tmpfile"
      },
      "enriched_text": {
        "status": "OK",
        "relations": [
          {
            "subject": {
              "sentiment": {
                "type": "positive",
                "mixed": false,
                "score": 0.635621
              },
              "text": "Obi-Wan's investigation",
              "keywords": [
                {
                  "text": "Obi-Wan"
                },
                {
                  "text": "investigation"
                }
              ],
              "entities": [
                {
                  "type": "Person",
                  "text": "Obi-Wan"
                }
              ]
            },
            "action": {
              "lemmatized": "lead",
              "verb": {
                "tense": "pre

### Process returned results

If you properly structure the query, Watson is able to do a pretty good job of finding the relevant information. But the result returned is a JSON object. Now your task is to convert that result into an appropriate response that best addresses the original natural language question that was asked.

E.g. if the question was "Who saved Han Solo from Jabba the Hutt?" the answer should ideally just be "The Rebels" and not the entire paragraph describing Han Solo's rescue. But that can be a backup response if you cannot be more specific.

_**Note**: You may have to go back to the previous step and modify the query, especially what you want the Discovery service to return, and this may depend on the intent / dialog node triggered. E.g. study the different parts of a "relation" structure to see how you might construct queries to match them._

In [112]:
# TODO: Extract the entities found in the question text
for result in query_results['results']:
    for relation in result['enriched_text']['relations']:
        #print(relation['sentence'])
        if 'vow' in relation['action']['lemmatized']:
            entities = subject.get('entities', None)
            #print(entities)
            for e in entities:
                if e["type"] == "Person":
                    print ("Answer: "+e["text"])
                    print("Explanation: "+relation['sentence'])

Answer: Qui-Gon
Explanation:  Undaunted, Qui-Gon vows to train Anakin himself.


## 5. Reflections

**Q**: Now that you have gone through this exercise of designing a system that uses two IBM Watson services, what did you learn? What were some of the strengths and weaknesses of this approach?

**A**: Watsons language parsing seems to be quiet successful. Enrichment and meta data is also good information. There's much that can be done with right processing of the results. The hard coding of dialogues and answers takes away from the true artificial intelligence we came to expect. Maybe there could be more than there. 


## (Optional) Extensions

We have provided a set of sample data files containing Star Wars plot summaries. But as mentioned before, you are free to use your own dataset. In fact, a larger dataset maybe more suitable for use with IBM Watson's NLP services. If you used your own dataset, answer the following questions.

**Q**: What dataset did you use, and in what ways is it different from the sample files provided?

**A**:


**Q**: Either include your dataset in the .zip file or repository you submit, or provide clear instructions on how to obtain the dataset, so that your reviewer can run your notebook or inspect the data to verify your results.

**A**: 


_You can also design a web-based application that utilizes these services and deploy that on Bluemix! If you do, please share with your instructors and peers._